# Dependencies & Read Data

In [1]:
#Imports
import pandas as pd
import requests
import gmaps
import re
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
#Read the CSV & Create DF
facilities_w_ratings_df = pd.read_csv("facility_data_with_ratings.csv")
facilities_w_ratings_df.head(10)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,origin_lat,origin_long,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189746,-105.102332,40.189772,-105.102667,OPERATIONAL,Runza Restaurant,4.2,1.0,210.0
1,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST BOULDER CO USA,RESTAURANT MORE THAN 200 SEATS,FULL SERVICE FULL MENU,40.016235,-105.282759,40.016553,-105.282845,OPERATIONAL,Frank’s Chophouse,4.3,3.0,446.0
2,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,NaN,NaN,39.963180,-105.509594,NaN,NaN,NaN,NaN,NaN
3,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163078,-105.098175,40.163077,-105.098557,NaN,NaN,NaN,NaN,NaN
4,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD BOULDER CO USA,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,40.072805,-105.201040,40.072171,-105.200873,OPERATIONAL,Tod's Espresso Cafe,4.3,1.0,170.0
5,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175326,-105.102459,40.175377,-105.102119,NaN,NaN,NaN,NaN,NaN
6,FA0002790,PALETERIA LA MICHOACANA,2130 N MAIN ST UNIT 1 LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,40.196791,-105.102409,40.196846,-105.100961,OPERATIONAL,Paleteria La Michoacana,4.0,1.0,164.0
7,FA0001363,TIBETS RESTAURANT AND BAR,321 S MCCASLIN BLVD LOUISVILLE CO USA,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,39.966720,-105.163530,39.966605,-105.163988,OPERATIONAL,Tibet's Restaurant & Bar,4.4,2.0,293.0
8,FA0000878,UTE CREEK GOLF COURSE,2000 UTE CREEK DR LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,40.195844,-105.067285,40.189249,-105.063881,NaN,NaN,NaN,NaN,NaN
9,FA0004983,LEVY IPF 1 (MOBILE),2400 COLORADO AVE BOULDER CO USA,MOBILE UNIT FULL FOOD SERVICE,MOBILE UNITS,40.008231,-105.264147,40.009191,-105.266914,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [35]:
# Dropping Original data's lat/long
facilities_w_ratings_df.drop(['origin_lat','origin_long'], inplace=True,axis=1)
facilities_w_ratings_df.head()

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189772,-105.102667,OPERATIONAL,Runza Restaurant,4.2,1.0,210.0
1,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST BOULDER CO USA,RESTAURANT MORE THAN 200 SEATS,FULL SERVICE FULL MENU,40.016553,-105.282845,OPERATIONAL,Frank’s Chophouse,4.3,3.0,446.0
2,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,NaN,NaN,NaN,NaN,NaN
3,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163077,-105.098557,NaN,NaN,NaN,NaN,NaN
4,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD BOULDER CO USA,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,40.072171,-105.200873,OPERATIONAL,Tod's Espresso Cafe,4.3,1.0,170.0


In [45]:
# drop null values
valid_facilities = facilities_w_ratings_df.dropna()
valid_facilities.head(3)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189772,-105.102667,OPERATIONAL,Runza Restaurant,4.2,1.0,210.0
1,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST BOULDER CO USA,RESTAURANT MORE THAN 200 SEATS,FULL SERVICE FULL MENU,40.016553,-105.282845,OPERATIONAL,Frank’s Chophouse,4.3,3.0,446.0
4,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD BOULDER CO USA,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,40.072171,-105.200873,OPERATIONAL,Tod's Espresso Cafe,4.3,1.0,170.0


In [46]:
# drop all columns except for facility id and rating data
valid_facilities = valid_facilities[['facilityId', 'biz_status', 'rating', 'price_level', 'total_user_ratings']]
valid_facilities.head(3)

,facilityId,biz_status,rating,price_level,total_user_ratings
0,FA0003323,OPERATIONAL,4.2,1.0,210.0
1,FA0000616,OPERATIONAL,4.3,3.0,446.0
4,FA0003472,OPERATIONAL,4.3,1.0,170.0


In [47]:
# fill NaN rows with zeroes
invalid_facilities = facilities_w_ratings_df.fillna(0)
invalid_facilities.head(3)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189772,-105.102667,OPERATIONAL,Runza Restaurant,4.2,1.0,210.0
1,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST BOULDER CO USA,RESTAURANT MORE THAN 200 SEATS,FULL SERVICE FULL MENU,40.016553,-105.282845,OPERATIONAL,Frank’s Chophouse,4.3,3.0,446.0
2,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,0,0,0.0,0.0,0.0


In [48]:
# use 0 values to create a new dataframe for reexamination
invalid_facilities = invalid_facilities.loc[invalid_facilities["biz_status"] == 0]
invalid_facilities.head(3)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
2,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,0,0,0.0,0.0,0.0
3,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163077,-105.098557,0,0,0.0,0.0,0.0
5,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175377,-105.102119,0,0,0.0,0.0,0.0


# Data Exports

In [49]:
# Export facility dataset w/ Google Data for 
output_data = "db_facility_data_w_ratings.csv"
valid_facilities.to_csv(output_data,header=True,index=False)

In [50]:
# Export unique address dataset w/ Google Data
output_data = "invalid_facility_data.csv"
invalid_facilities.to_csv(output_data,header=True,index=False)